In [3]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6 

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 4.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=2d023bc4ebbcd3869d2e28fd647dad2c8de9ca7a303ef689d47fa2506004dea3
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle -v

Kaggle API 1.5.6


In [5]:
!kaggle competitions download -c logistic-classification-diabetes

  0% 0.00/16.3k [00:00<?, ?B/s]
100% 16.3k/16.3k [00:00<00:00, 16.6MB/s]


In [6]:
!unzip logistic-classification-diabetes

Archive:  logistic-classification-diabetes.zip
  inflating: submission_form.csv     
  inflating: test_data.csv           
  inflating: train.csv               


In [7]:
import torch
import torch.optim as optim 
import torch.nn.functional as F 
# 소프트맥스 함수를 사용하기 위함. 
import numpy as np 
# numpy 
import pandas as pd
# pandas

torch.manual_seed(1)

data_diabetes = pd.read_csv('train.csv')
data_diabetes.head()

x_data = data_diabetes.iloc[:,1:9]
#x_data.head()

y_data = data_diabetes.iloc[:,[-1]]
#y_data.head()

print(x_data)
print(y_data)

x_train = torch.FloatTensor(x_data.values)
y_train = torch.FloatTensor(y_data.values)

print(x_train.shape)
print(y_train.shape)

            0         1         2  ...         5         6         7
0   -0.294118  0.487437  0.180328  ...  0.001490 -0.531170 -0.033333
1   -0.882353 -0.145729  0.081967  ... -0.207153 -0.766866 -0.666667
2   -0.058824  0.839196  0.049180  ... -0.305514 -0.492741 -0.633333
3   -0.882353 -0.105528  0.081967  ... -0.162444 -0.923997  0.000000
4    0.000000  0.376884 -0.344262  ...  0.284650  0.887276 -0.600000
..        ...       ...       ...  ...       ...       ...       ...
704  0.000000  0.346734 -0.049180  ... -0.213115 -0.766012  0.000000
705 -0.647059  0.025126  0.213115  ... -0.120715 -0.963279 -0.633333
706 -0.176471  0.879397 -0.180328  ...  0.010432 -0.361230 -0.566667
707 -0.647059  0.738693  0.278689  ...  0.007452 -0.238258 -0.666667
708  0.176471 -0.055276  0.180328  ... -0.311475 -0.558497  0.166667

[709 rows x 8 columns]
       8
0    0.0
1    1.0
2    0.0
3    1.0
4    0.0
..   ...
704  1.0
705  1.0
706  0.0
707  0.0
708  1.0

[709 rows x 1 columns]
torch.Size([709,

In [96]:
w = torch.zeros((8, 1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([w,b], lr = 1e-3)

nb_epochs = 50000

for epoch in range(nb_epochs+1):

    hypothesis = torch.sigmoid(x_train.matmul(w)+ b) 

    cost = -(y_train * torch.log(hypothesis) + (1-y_train) * torch.log(1-hypothesis)).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10000 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('epoch {:4d}/{} cost : {:.6f} accuracy of {:2.2f}%'.format(
            epoch, nb_epochs,cost.item(),accuracy * 100
        ))

epoch    0/50000 cost : 0.693147 accuracy of 65.59%
epoch 10000/50000 cost : 0.572608 accuracy of 67.00%
epoch 20000/50000 cost : 0.539725 accuracy of 72.07%
epoch 30000/50000 cost : 0.520002 accuracy of 76.30%
epoch 40000/50000 cost : 0.507350 accuracy of 76.87%
epoch 50000/50000 cost : 0.498798 accuracy of 77.15%


In [97]:
hypothesis = torch.sigmoid(x_train.matmul(w) + b) # or .mm or @
#print(hypothesis)

prediction = hypothesis >= torch.FloatTensor([0.5])
#print(prediction)
#print(y_train)

# 정확도 계산 
correct_prediction = prediction.float() == y_train
#print(correct_prediction)

accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))


The model has an accuracy of 77.15% for the training set.


In [98]:
test = pd.read_csv('test_data.csv',header=None,skiprows=[0, 0])
x_data = test.loc[:,1:8]
#print(x_data)
x_data = np.array(x_data)
x_test = torch.FloatTensor(x_data)

hypothesis = torch.sigmoid(x_test.matmul(w)+b)
prediction = (hypothesis >= torch.FloatTensor([0.5])).int()
#if prediction == True: print(0)
print(prediction)


tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]], dtype=torch.int32)


In [13]:
len(prediction)

50

In [0]:
prediction = prediction.detach().numpy().reshape(-1,1)

id = np.array([i for i in range(len(prediction))]).reshape(-1,1)
result = np.hstack([id,prediction])
df = pd.DataFrame(result,columns=["ID","Label"])
df.to_csv("submission_form.csv",index=False,header=True)


In [100]:
result

array([[ 0,  1],
       [ 1,  1],
       [ 2,  1],
       [ 3,  1],
       [ 4,  1],
       [ 5,  1],
       [ 6,  1],
       [ 7,  1],
       [ 8,  1],
       [ 9,  1],
       [10,  1],
       [11,  1],
       [12,  1],
       [13,  1],
       [14,  0],
       [15,  1],
       [16,  1],
       [17,  1],
       [18,  1],
       [19,  1],
       [20,  1],
       [21,  1],
       [22,  0],
       [23,  1],
       [24,  1],
       [25,  0],
       [26,  0],
       [27,  1],
       [28,  0],
       [29,  1],
       [30,  0],
       [31,  0],
       [32,  1],
       [33,  1],
       [34,  1],
       [35,  0],
       [36,  0],
       [37,  1],
       [38,  1],
       [39,  1],
       [40,  1],
       [41,  0],
       [42,  1],
       [43,  0],
       [44,  1],
       [45,  1],
       [46,  1],
       [47,  1],
       [48,  1],
       [49,  1]])

In [101]:
!kaggle competitions submit -c logistic-classification-diabetes -f submission_form.csv -m "14010974_이기택"

100% 249/249 [00:02<00:00, 102B/s]
Successfully submitted to logistic classification : diabetes